In [8]:
#!wget -nc 'https://bora.uib.no/bora-xmlui/bitstream/handle/1956/10699/MUFI-CodeChart-4-0.pdf?isAllowed=y&sequence=4' && mv 'MUFI-CodeChart-4-0.pdf?isAllowed=y&sequence=4' ./MUFI-CodeChart-4-0.pdf
#!pip install pdfplumber
overwrite_source_code = False

In [2]:
import pdfplumber
import re

code_points = []
#pattern = re.compile(r'\b([0-9A-F]{4,5})\b\S[A-Z]{3,20}\S')
pattern = re.compile(r'([0-9A-F]{4,5})\s*[A-Z\-\s]{5,20}')

with pdfplumber.open("MUFI-CodeChart-4-0.pdf") as pdf:
    match_count = 0
    accepted_count = 0
    for page_n, page in enumerate(pdf.pages):
        text = page.extract_text()
        codes_of_page = pattern.findall(text)
        code_points.extend(codes_of_page)
        if page_n == 12000:
            print(repr(text))
            print('-----------------')
            print(pattern.findall(text))
            break
        print(f"page {page_n}: {len(codes_of_page)} {','.join(codes_of_page)}")

page 0: 0 
page 1: 0 
page 2: 0 
page 3: 0 
page 4: 0 
page 5: 0 
page 6: 0 
page 7: 0 
page 8: 0 
page 9: 0 
page 10: 0 
page 11: 7 0020,0021,0022,0023,0024,0025,0026
page 12: 13 0027,02BC,0028,0029,002A,002B,002C,002D,2010,2212,002E,002E,002F
page 13: 13 0030,0031,0032,0033,0034,0035,0036,0037,0038,0039,003A,003B,003B
page 14: 12 003C,003D,003E,003F,0040,0041,0042,0043,0044,0045,0046,0047
page 15: 14 0048,0049,004A,004B,004C,004D,004E,004F,0050,0051,0052,0053,0054,0055
page 16: 14 0056,0057,0058,0059,005A,005B,005C,005D,005E,005F,0060,0061,0062,0063
page 17: 14 0064,0065,0066,0067,0068,0069,006A,006B,006C,006D,006E,006F,0070,0071
page 18: 14 0072,0073,0074,0075,0076,0077,0078,0079,007A,007B,007C,007D,007E,007F
page 19: 9 00A0,00A1,00A2,00A3,00A4,00A5,00A6,00A7,00A8
page 20: 13 00A9,00AA,00AB,00AC,00AD,00AE,00AF,00B0,00B1,00B2,00B3,00B4,00B5
page 21: 13 00B6,00B7,00B8,00B9,00BA,00BB,00BC,00BD,00BE,00BF,00C0,00C1,00C2
page 22: 14 00C3,00C4,00C5,00C6,00C7,00C8,00C9,00CA,00CB,00CC,00CD,0

In [3]:
from collections import defaultdict
code_point_occ = defaultdict(lambda: 0)

for code_point in code_points:
    code_point_occ[code_point]+= 1

print("Duplicates for verification in the pdf")
for n, (code, occ) in enumerate(code_point_occ.items()):
    if occ>1:
        print(f"{code}: {occ} ", end = "" if n % 10 else "\n")

Duplicates for verification in the pdf
0027: 2 02BC: 2 002D: 2 2010: 2 2212: 2 002E: 3 0030: 2 0031: 2 
0032: 2 0033: 2 0034: 2 0035: 2 0036: 2 0037: 2 0038: 2 0039: 2 003B: 3 
0041: 2 0042: 2 0043: 2 0044: 2 0045: 2 
0046: 2 0047: 2 0048: 2 0049: 2 004A: 2 004B: 2 004C: 2 004D: 2 004E: 2 004F: 2 
0050: 2 0051: 2 0052: 2 0053: 2 0054: 2 0055: 2 0056: 2 0057: 2 0058: 2 0059: 2 
005A: 2 00B7: 2 00BA: 2 00C0: 2 00C1: 2 00C2: 2 00C3: 2 00C4: 2 00C5: 2 00C6: 2 00C7: 2 00C8: 2 00C9: 2 
00CA: 2 00CB: 2 00CC: 2 00CD: 2 00CE: 2 00CF: 2 00D1: 2 00D2: 2 00D3: 2 
00D4: 2 00D5: 2 00D6: 2 00D8: 2 00D9: 2 00DA: 2 00DB: 2 00DC: 2 00DE: 2 00DF: 2 00F0: 3 00FE: 2 014B: 2 0152: 2 0178: 2 0199: 2 019C: 3 
01B7: 2 0254: 2 0266: 2 026F: 3 0292: 2 0301: 2 0303: 2 0304: 4 0305: 2 0307: 2 0366: 2 
1D35: 2 1D6B: 2 1DCE: 2 2020: 2 
2021: 2 F1D2: 3 2025: 2 204A: 2 
2056: 2 F2F0: 2 20B0: 3 2114: 2 211E: 2 211F: 2 2125: 2 
2183: 2 2C7D: 3 
2E40: 2 A778: 2 AB57: 2 F1BB: 2 E8DE: 2 E8DF: 2 E8C8: 2 E8C9: 2 E8C6: 2 E8C7

In [6]:
public_bmp_mufi_chr = []
public_nonbmp_mufi_chr = []
private_bmp_mufi_chr = []
private_nonbmp_mufi_chr = []

pua_code_ranges = [[0xe000, 0xf8ff], [0xf0000, 0xffffd], [0x100000, 0x10ffd]]

for code_point_hex_str in sorted(set(code_points)):
    hex_code = int(code_point_hex_str, 16)
    if any([pua[0] <= hex_code <= pua[1] for pua in pua_code_ranges]):
        if hex_code < 65536:
            private_bmp_mufi_chr.append(chr(hex_code))
        else:
            private_nonbmp_mufi_chr.append(chr(hex_code))
    else:
        if hex_code < 65536:
            public_bmp_mufi_chr.append(chr(hex_code))
        else:
            public_nonbmp_mufi_chr.append(chr(hex_code))


public_bmp_mufi_str = ''.join(public_bmp_mufi_chr)
public_nonbmp_mufi_str = ''.join(public_nonbmp_mufi_chr)
private_bmp_mufi_str = ''.join(private_bmp_mufi_chr)
private_nonbmp_mufi_str = ''.join(private_nonbmp_mufi_chr)

print(f"public bmp mufi characters length: {len(public_bmp_mufi_str)}")
print(f"public nonbmp mufi characters length: {len(public_nonbmp_mufi_str)}")
print(f"private bmp mufi characters length: {len(private_bmp_mufi_str)}")
print(f"private nonbmp mufi characters length: {len(private_nonbmp_mufi_str)}")

public bmp mufi characters length: 777
public nonbmp mufi characters length: 11
private bmp mufi characters length: 713
private nonbmp mufi characters length: 0


In [7]:
def format_string_literal(input_string, line_length=20):
    if len(input_string) == 0:
        return repr(input_string)
    else:
        chunks = [input_string[i:i+line_length] for i in range(0, len(input_string), line_length)]
        return ' + \\\n'.join(f'{repr(chunk)}' for chunk in chunks)
#print(format_string_literal(normal_mufi_str))
#print(format_string_literal(pua_mufi_str))
src_str = "# code automatically generated look at experiments/mufi_extraction/mufi_extractor.ipynb\n\n"

src_str += "__public_bmp_mufi_str = "
src_str += format_string_literal(public_bmp_mufi_str)
src_str += "\n\n"

src_str += "__public_nonbmp_mufi_str = "
src_str += format_string_literal(public_nonbmp_mufi_str)
src_str += "\n\n"

src_str += "__private_bmp_mufi_str = "
src_str += format_string_literal(private_bmp_mufi_str)
src_str += "\n\n"

src_str += "__private_nonbmp_mufi_str = "
src_str += format_string_literal(private_nonbmp_mufi_str)
src_str += "\n\n"

src_str += "# PUBLIC = PUBLIC\n# PRIVATE = PUBLIC U PRIVATE\n"
src_str += "# BMP = BMP\n# NONBMP = BMP U NONBMP\n\n"

src_str += "all_bmp_mufi = {'bmp_mufi': __public_bmp_mufi_str, 'bmp_pua_mufi': __public_bmp_mufi_str +  __private_bmp_mufi_str}\n"
src_str += "all_nonbmp_mufi = {k:v for k, v in all_bmp_mufi.items()}\n"
src_str += "all_nonbmp_mufi.update({'nonbmp_mufi': __public_nonbmp_mufi_str, 'nonbmp_pua_mufi': __public_nonbmp_mufi_str +  __private_nonbmp_mufi_str})\n"

if overwrite_source_code == True:
    with open("../../pylelemmatize/map_mufi.py", "w") as f:
        f.write(src_str)

print(src_str)


# code automatically generated look at experiments/mufi_extraction/mufi_extractor.ipynb

__public_bmp_mufi_str = ' !"#$%&\'()*+,-./0123' + \
'456789:;<=>?@ABCDEFG' + \
'HIJKLMNOPQRSTUVWXYZ[' + \
'\\]^_`abcdefghijklmno' + \
'pqrstuvwxyz{|}~\x7f\xa0¡¢£' + \
'¤¥¦§¨©ª«¬\xad®¯°±²³´µ¶·' + \
'¸¹º»¼½¾¿ÀÁÂÃÄÅÆÇÈÉÊË' + \
'ÌÍÎÏÐÑÒÓÔÕÖ×ØÙÚÛÜÝÞß' + \
'àáâãäåæçèéêëìíîïðñòó' + \
'ôõö÷øùúûüýþÿĀāĂăĄąĆć' + \
'ĊċĐđĒēĔĕĖėĘęĠġħĪīĬĭĮ' + \
'įİıĲĳĹĺŁłŃńŊŋŌōŎŏŐőŒ' + \
'œŔŕŚśŪūŬŭŮůŰűŲųŴŵŶŷŸ' + \
'ŻżſƀƕƙƚƜƞƦƵƶƷƿǑǒǓǔǕǖ' + \
'ǢǣǤǥǪǫǬǭǴǵǶǷǼǽǾǿȜȝȦȧ' + \
'ȪȫȮȯȲȳȷɈɉɔɖəɟɡɢɦɨɪɯɲ' + \
'ɴɶɼʀʉʏʒʙʜʟʻʼˈ˘˙˚˛˜˝ˣ' + \
'̧̨̣̲̳̀́̂̃̄̅̆̇̈̉̊̋̍̎̕' + \
'̶͙̾̿͂͛ͣͤͥͦͧͨͩͪͫͬͭͮͯ͜' + \
'Θθස჻ᚠᛘᴀᴁᴄᴅᴆᴇᴊᴋᴍᴏᴘᴛᴜᴠ' + \
'ᴡᴢᴵᵫᵹ᷐᷎᷏᷑᷒ᷓᷔᷕᷖᷗᷘᷙᷚᷛ᷍' + \
'ᷜᷝᷞᷟᷠᷡᷢᷣᷤᷥᷦḂḃḄḅḊḋḌḍḖ' + \
'ḗḞḟḢḣḤḥḰḱḲḳḶḷḾḿṀṁṂṃṄ' + \
'ṅṆṇṒṓṔṕṖṗṘṙṚṛṠṡṢṣṪṫṬ' + \
'ṭṾṿẀẁẂẃẄẅẆẇẈẉẎẏẒẓẘẙẜ' + \
'ẝẞẟẠạẢảẮắẸẹỈỉỊịỌọỎỏỤ' + \
'ụỦủỲỳỴỵỶỷỺỻỼỽỾỿ\u2000\u2001\u2002\u2003\u2004' + \
'\u2005\u2006\u2007\u2008\u2009\u200a\u200b‐‑‒–—―‖‘’‚‛“”' + \
'„‟†‡•‣․‥…‧\u202f‰′″‹›※⁂⁄⁅' + \
'⁆⁊⁋⁖⁘⁜⁰⁴⁵⁶⁷⁸⁹₀₁₂₃₄₅₆' + \
'